In [1]:
""" Demand data is provided as volumes. Calculate Fluxes.
-------------------------------------------------------------------------------
PCRGLOBWB Data for demand is provided in volumes with units millionm3 
(implicit per pixel/per time step); converting to fluxes (m) and storing to 
earth engine asset. 

Notes:
the uplaod API was overloading (>3000tasks submitted). A Retrying module has 
been added to the script as well as a check if the files exists. Some tasks 
will fail due to asynchronous submission.

Converts demand to flux

Author: Rutger Hofste
Date: 20180418
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    SCRIPT_NAME (string) : Script name
    EE_VERSION (integer) : Earth Engine path version.
    OUTPUT_VERSION (integer) : Output version.
    TESTING (boolean) : Testing.

Returns:

"""

# Input Parameters

SCRIPT_NAME = "Y2018M04D18_RH_Demand_Fluxes_5min_EE_V01"
EE_VERSION = 9

OUTPUT_VERSION = 2 

TESTING = 0

ee_path = "projects/WRI-Aquaduct/PCRGlobWB20V{:02.0f}".format(EE_VERSION)


print("Input ee: " + ee_path +
      "\nOutput ee: " + ee_path )

Input ee: projects/WRI-Aquaduct/PCRGlobWB20V09
Output ee: projects/WRI-Aquaduct/PCRGlobWB20V09


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M04D22 UTC 13:07


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import subprocess
import pandas as pd
import ee
import re
from retrying import retry
from datetime import timedelta
import aqueduct3

ee.Initialize()

In [4]:

def update_property_script_used(image):
    image = image.set("script_used",SCRIPT_NAME)
    return image

def update_property_output_version(image):
    image = image.set("output_version",OUTPUT_VERSION)
    return image

In [5]:
old_unit = "millionm3"
new_unit = "m"

sectors = ["PDom","PInd","PIrr","PLiv"]
demand_types = ["WW","WN"]
temporal_resolutions = ["year","month"]

if TESTING:
    sectors = ["PDom"]
    demand_types = ["WW"]
    temporal_resolutions = ["year"]

def ic_volume_to_flux_5min_millionm3_m2(ic_input_asset_id,output_version):
    """ Convert an imagecollection from volume to flux.
    -------------------------------------------------------------------------------
    The result is stored in an imagecollection with the same name as the input
    imagecollection but with 'm' instead of 'millionm3'.
    
    Input ic:
    global_historical_PDomWW_year_millionm3_5min_1960_2014
    
    Output ic:    
    global_historical_PDomWW_year_m_5min_1960_2014
    
    Args:
        ic_input_asset_id (string) : asset id of input imagecollection.

    """
    start_time = time.time()
    df_errors = pd.DataFrame()
    
    df = aqueduct3.earthengine.get_df_from_ic(ic_input_asset_id)
    df = aqueduct3.earthengine.add_export_parameters_for_export(df,old_unit,new_unit,output_version)
    
    # Creating ImageCollection(s)
    output_ic_asset_ids = list(df["output_ic_asset_id"].unique())
    for output_ic_asset_id in output_ic_asset_ids:
        command, result = aqueduct3.earthengine.create_imageCollection(output_ic_asset_id)
        print(command,result)
    
    
    # Bacth Converting and uploading.     
    for index, row in df.iterrows():
        elapsed_time = time.time() - start_time 
        print("Index: {:04.0f} Elapsed: {}".format(index, timedelta(seconds=elapsed_time)))
    
        description = row["description"]
        output_image_asset_id = row["output_image_asset_id"]
    
    
        if aqueduct3.earthengine.asset_exists(output_image_asset_id):
            print("Asset exists, skipping: {}".format(output_image_asset_id))
        else:
            i_volume_millionm3_5min = ee.Image(row["input_image_asset_id"])
            i_flux_m_5min = aqueduct3.earthengine.volume_to_flux_5min_millionm3_m2(i_volume_millionm3_5min)
            i_flux_m_5min = update_property_script_used(i_flux_m_5min)
            i_flux_m_5min = update_property_output_version(i_flux_m_5min)
            

            aqueduct3.earthengine.export_image_global_5min(i_flux_m_5min,description,output_image_asset_id)
            print(output_image_asset_id)    
    
    return df


In [ ]:
df = pd.DataFrame()
for sector in sectors:
    for demand_type in demand_types:
        for temporal_resolution in temporal_resolutions:
            print(sector,demand_type,temporal_resolution)
            ic_input_file_name = "global_historical_{}{}_{}_millionm3_5min_1960_2014".format(sector,demand_type,temporal_resolution)
            ic_input_asset_id = "{}/{}".format(ee_path,ic_input_file_name)
            df = ic_volume_to_flux_5min_millionm3_m2(ic_input_asset_id,OUTPUT_VERSION)
            

PDom WW year
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014 b'Asset projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014 already exists\n'
Index: 0000 Elapsed: 0:00:04.001557
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I000Y1960M12
Index: 0001 Elapsed: 0:00:06.073905
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I001Y1961M12
Index: 0002 Elapsed: 0:00:08.138236
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I002Y1962M12
Index: 0003 Elapsed: 0:00:10.188677
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_

Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I039Y1999M12
Index: 0040 Elapsed: 0:01:23.728795
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I040Y2000M12
Index: 0041 Elapsed: 0:01:25.806520
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I041Y2001M12
Index: 0042 Elapsed: 0:01:27.855082
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I042Y2002M12
Index: 0043 Elapsed: 0:01:29.900005
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I043Y2003M12
Index: 0044 Elap

Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I023Y1961M12
Index: 0024 Elapsed: 0:00:52.497123
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I024Y1962M01
Index: 0025 Elapsed: 0:00:54.509609
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I025Y1962M02
Index: 0026 Elapsed: 0:00:56.503017
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I026Y1962M03
Index: 0027 Elapsed: 0:00:58.499717
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I027Y1962M04
Index:

Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I063Y1965M04
Index: 0064 Elapsed: 0:02:12.027540
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I064Y1965M05
Index: 0065 Elapsed: 0:02:14.144342
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I065Y1965M06
Index: 0066 Elapsed: 0:02:16.343759
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I066Y1965M07
Index: 0067 Elapsed: 0:02:18.481879
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I067Y1965M08
Index:

Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I103Y1968M08
Index: 0104 Elapsed: 0:03:34.567079
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I104Y1968M09
Index: 0105 Elapsed: 0:03:36.576614
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I105Y1968M10
Index: 0106 Elapsed: 0:03:38.577502
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I106Y1968M11
Index: 0107 Elapsed: 0:03:40.579236
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I107Y1968M12
Index:

...

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I472Y1999M05
Index: 0473 Elapsed: 0:28:46.346543
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I473Y1999M06
Index: 0474 Elapsed: 0:28:50.090038
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I474Y1999M07
Index: 0475 Elapsed: 0:28:53.828075
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I475Y1999M08
Index: 0476 Elapsed: 0:28:57.469526
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I476Y1999M09
Index: 0477 Elapsed: 0:29:01.076960
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/globa

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I518Y2003M03
Index: 0519 Elapsed: 0:31:38.722029
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I519Y2003M04
Index: 0520 Elapsed: 0:31:42.543063
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I520Y2003M05
Index: 0521 Elapsed: 0:31:46.484241
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I521Y2003M06
Index: 0522 Elapsed: 0:31:50.355372
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I522Y2003M07
Index: 0523 Elapsed: 0:31:54.275255
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/globa

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I564Y2007M01
Index: 0565 Elapsed: 0:34:29.697634
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I565Y2007M02
Index: 0566 Elapsed: 0:34:33.373383
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I566Y2007M03
Index: 0567 Elapsed: 0:34:37.097719
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I567Y2007M04
Index: 0568 Elapsed: 0:34:40.810993
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I568Y2007M05
Index: 0569 Elapsed: 0:34:44.482805
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/globa

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I610Y2010M11
Index: 0611 Elapsed: 0:37:17.797492
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I611Y2010M12
Index: 0612 Elapsed: 0:37:21.434173
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I612Y2011M01
Index: 0613 Elapsed: 0:37:25.049172
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I613Y2011M02
Index: 0614 Elapsed: 0:37:28.700630
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I614Y2011M03
Index: 0615 Elapsed: 0:37:31.853838
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/globa

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I656Y2014M09
Index: 0657 Elapsed: 0:40:03.615346
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I657Y2014M10
Index: 0658 Elapsed: 0:40:07.246346
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I658Y2014M11
Index: 0659 Elapsed: 0:40:10.915224


In [9]:
df.head()

,input_image_asset_id,input_ic_asset_id,output_ic_asset_id,output_image_asset_id,description
0,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_PLivWN_month_m_5min_1960_201...
1,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_PLivWN_month_m_5min_1960_201...
2,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_PLivWN_month_m_5min_1960_201...
3,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_PLivWN_month_m_5min_1960_201...
4,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_PLivWN_month_m_5min_1960_201...


In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

Previous runs:  
0:40:10.915224